In [6]:
from pdp_utils import *
from tqdm import tqdm
import time
import random
import numpy as np
n = 10000

def oneopt_operator_nborhood(cur_sol):
    nbors = []
    for i in range(len(cur_sol)):
        currentlist = []
        for j in range(len(cur_sol)):
            insertable = cur_sol.copy()
            item_to_insert = insertable[i]
            insertable = insertable[:i] + insertable[i+1:]
            insertable.insert(j, item_to_insert)
            if solution_sanity(insertable):
                currentlist.append(insertable)
        nbors = nbors + currentlist
    return nbors

def twoopt_operator_nborhood(cur_sol):
    
    nbors = []
    for i in range(len(cur_sol)):
        for j in range(len(cur_sol)):
            insertable = cur_sol.copy()
            i_val = insertable[i]
            insertable[i] = insertable[j]
            insertable[j] = i_val
            if solution_sanity(insertable):
                nbors.append(insertable)
    return nbors

def threeopt_operator_nborhood(cur_sol):
    nbors = []
    for i in range(len(cur_sol)):
        for j in range(len(cur_sol)):
            for k in range(len(cur_sol)):
                insertable = cur_sol.copy()
                i_val = insertable[i]
                j_val = insertable[j]
                insertable[i] = insertable[k]
                insertable[j] = i_val
                insertable[k] = j_val
                if solution_sanity(insertable):
                    nbors.append(insertable)
    return nbors

def solution_sanity(sol):
    """Ensures that any given solution both picks up and delivers its assigned calls"""
    currentcalls = []
    for i in range(len(sol)):
        if sol[i] == 0: 
            if len(currentcalls) > 0:
                return False
        else:
            if sol[i] in currentcalls:
                currentcalls.remove(sol[i])
            else:
                currentcalls.append(sol[i])
    return True

def rejected_calls_cost(sol, prob):
    total = 0
    sol = sol.copy()
    sol.reverse()
    for call in sol:
        if call == 0:
            return total
        total += prob["Cargo"][call - 1][3]
    raise ValueError("Bad solution")
        
    

In [10]:
def localsearch(init_sol, operator, prob):
    best_sol = init_sol
    best_sol_cost = cost_function(best_sol, prob) + rejected_calls_cost(best_sol, prob)
    for i in tqdm(range(n)):
        best_nbor = None
        best_nbor_cost = float('inf')
        nbors = operator(best_sol)
        for nbor in nbors:
            feasibility, c = feasibility_check(nbor, prob)
            if feasibility:
                nbor_cost = cost_function(nbor, prob) + rejected_calls_cost(nbor, prob)
                if nbor_cost < best_nbor_cost:
                    best_nbor_cost = nbor_cost
                    best_nbor = nbor
        
        if best_nbor is not None and best_sol_cost > best_nbor_cost:
            best_sol = best_nbor
            best_sol_cost = best_nbor_cost
        if best_nbor is not None and best_sol_cost < best_nbor_cost:
            # We've found local optima. Will not get better.
            break
    return best_sol, best_sol_cost


def annealing(init_sol, operator, prob):
    best_sol = init_sol
    best_sol_cost = cost_function(best_sol, prob) + rejected_calls_cost(best_sol, prob)
    delta_es = [0]
    
    for i in tqdm(range(100)):
        best_nbor = None
        best_nbor_cost = float('inf')
        nbors = operator(best_sol)
        for nbor in nbors:
            feasibility, c = feasibility_check(nbor, prob)
            if feasibility:
                nbor_cost = cost_function(nbor, prob) + rejected_calls_cost(nbor, prob)
                if nbor_cost < best_nbor_cost:
                    best_nbor_cost = nbor_cost
                    best_nbor = nbor
                else:
                    randval = random.random()
                    if randval < 0.8:
                        best_nbor_cost = nbor_cost
                        best_nbor = nbor
        
        if best_nbor is not None:
            delta_es.append(best_sol_cost - best_nbor_cost)
            if best_sol_cost > best_nbor_cost:
                best_sol = best_nbor
                best_sol_cost = best_nbor_cost
    T = (sum(delta_es) / len(delta_es)) / np.log(0.8)
    alpha = np.sqrt(0.1 / T)
    
    for i in tqdm(range(n - 100)):
        best_nbor = None
        best_nbor_cost = float('inf')
        nbors = operator(best_sol)
        for nbor in nbors:
            feasibility, c = feasibility_check(nbor, prob)
            if feasibility:
                nbor_cost = cost_function(nbor, prob) + rejected_calls_cost(nbor, prob)
                if nbor_cost < best_nbor_cost:
                    best_nbor_cost = nbor_cost
                    best_nbor = nbor
                else:
                    delta_e = nbor_cost - best_nbor_cost
                    randval = random.random()
                    if randval < np.exp(-delta_e / T):
                        best_nbor_cost = nbor_cost
                        best_nbor = nbor
        
        if best_nbor is not None:
            delta_es.append(best_sol_cost - best_nbor_cost)
            if best_sol_cost > best_nbor_cost:
                best_sol = best_nbor
                best_sol_cost = best_nbor_cost
        T = T * alpha

        
    return best_sol, best_sol_cost

In [11]:
def run_problem(problem, initial_solution):
    methods = [localsearch, annealing]
    operators = [oneopt_operator_nborhood, twoopt_operator_nborhood, threeopt_operator_nborhood]
    sols = []
    costs = []
    for method in methods:
        for operator in operators:
            best_sol, best_sol_cost = method(initial_solution, operator, problem)
            sols.append(best_sol)
            costs.append(best_sol_cost)
    return sols, costs

## C7V3

In [12]:
problem = load_problem('./Call_7_Vehicle_3.txt')
init_sol = [7, 7, 0, 3, 3, 0, 1, 5, 5, 1, 0, 6, 4, 2]
init_cost = cost_function(init_sol, problem) + rejected_calls_cost(init_sol, problem)
sols_iter = []
costs_iter = []
for i in range(10):
    sols, costs = run_problem(problem, init_sol)
    sols_iter.append(sols)
    costs_iter.append(costs)

best_local_1opt = min([op[0] for op in costs_iter])
print("Avg Local 1-opt", np.mean([op[0] for op in costs_iter]))
print("Best Local 1-opt", best_local_1opt)
print("Improvement%", (init_cost - best_local_1opt) / 100)

best_local_2opt = min([op[1] for op in costs_iter])
print("Avg Local 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Local 2-opt", best_local_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_local_3opt = min([op[2] for op in costs_iter])
print("Avg Local 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Local 3-opt", best_local_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 


best_anneal_1opt = min([op[0] for op in costs_iter])
print("Avg Anneal 1-opt", np.mean([op[0]for op in costs_iter]))
print("Best Anneal 1-opt", best_anneal_1opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_2opt = min([op[1] for op in costs_iter])
print("Avg Anneal 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Anneal 2-opt", best_anneal_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_3opt = min([op[2] for op in costs_iter])
print("Avg Anneal 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Anneal 3-opt", best_anneal_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 

 46%|████▌     | 4593/10000 [01:57<02:18, 38.98it/s]


KeyboardInterrupt: 

In [5]:
print(costs_iter)

[[2292765.5, 2292765.5, 705704.0, 2292765.5, 2292765.5, 705704.0], [2292765.5, 2292765.5, 705704.0, 2292765.5, 2292765.5, 705704.0], [2292765.5, 2292765.5, 705704.0, 2292765.5, 2292765.5, 705704.0], [2292765.5, 2292765.5, 705704.0, 2292765.5, 2292765.5, 705704.0], [2292765.5, 2292765.5, 705704.0, 2292765.5, 2292765.5, 705704.0], [2292765.5, 2292765.5, 705704.0, 2292765.5, 2292765.5, 705704.0], [2292765.5, 2292765.5, 705704.0, 2292765.5, 2292765.5, 705704.0]]


## C18V5

In [ ]:
problem = load_problem('./Call_18_Vehicle_5.txt')
init_sol = [0, 18, 18, 0, 14, 14, 0, 0, 0, 5, 9, 7, 13, 6, 11, 16, 8, 10, 4, 1, 17, 2, 15, 3, 12]
init_cost = cost_function(init_sol, problem) + rejected_calls_cost(init_sol, problem)
sols_iter = []
costs_iter = []
for i in range(1):
    sols, costs = run_problem(problem, init_sol)
    sols_iter.append(sols)
    costs_iter.append(costs)

best_local_1opt = min([op[0] for op in costs_iter])
print("Avg Local 1-opt", np.mean([op[0] for op in costs_iter]))
print("Best Local 1-opt", best_local_1opt)
print("Improvement%", (init_cost - best_local_1opt) / 100)

best_local_2opt = min([op[1] for op in costs_iter])
print("Avg Local 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Local 2-opt", best_local_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_local_3opt = min([op[2] for op in costs_iter])
print("Avg Local 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Local 3-opt", best_local_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 


best_anneal_1opt = min([op[0] for op in costs_iter])
print("Avg Anneal 1-opt", np.mean([op[0]for op in costs_iter]))
print("Best Anneal 1-opt", best_anneal_1opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_2opt = min([op[1] for op in costs_iter])
print("Avg Anneal 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Anneal 2-opt", best_anneal_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_3opt = min([op[2] for op in costs_iter])
print("Avg Anneal 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Anneal 3-opt", best_anneal_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 

## C35V7

In [ ]:
problem = load_problem('./Call_35_Vehicle_7.txt')
init_sol = [0, 0, 0, 31, 0, 0, 0, 0, 15, 19, 26, 32, 8, 1, 33, 5, 9, 20, 2, 34, 4, 21, 28, 3, 16, 13, 31, 7, 29, 30, 24, 14, 12, 6, 35, 25, 18, 23, 27, 17, 11, 10, 22]
init_cost = cost_function(init_sol, problem) + rejected_calls_cost(init_sol, problem)
sols_iter = []
costs_iter = []
for i in range(1):
    sols, costs = run_problem(problem, init_sol)
    sols_iter.append(sols)
    costs_iter.append(costs)

best_local_1opt = min([op[0] for op in costs_iter])
print("Avg Local 1-opt", np.mean([op[0] for op in costs_iter]))
print("Best Local 1-opt", best_local_1opt)
print("Improvement%", (init_cost - best_local_1opt) / 100)

best_local_2opt = min([op[1] for op in costs_iter])
print("Avg Local 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Local 2-opt", best_local_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_local_3opt = min([op[2] for op in costs_iter])
print("Avg Local 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Local 3-opt", best_local_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 


best_anneal_1opt = min([op[0] for op in costs_iter])
print("Avg Anneal 1-opt", np.mean([op[0]for op in costs_iter]))
print("Best Anneal 1-opt", best_anneal_1opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_2opt = min([op[1] for op in costs_iter])
print("Avg Anneal 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Anneal 2-opt", best_anneal_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_3opt = min([op[2] for op in costs_iter])
print("Avg Anneal 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Anneal 3-opt", best_anneal_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 

## C80V20

In [ ]:
problem = load_problem('./Call_80_Vehicle_20')
init_sol = [0, 51, 51, 0, 0, 0, 0, 0, 1, 1, 0, 34, 34, 46, 36, 36, 46, 0, 0, 65, 65, 0, 0, 0, 0, 0, 0, 0, 0, 21, 21, 0, 0, 0, 77, 44, 26, 70, 42, 20, 32, 56, 71, 48, 80, 79, 64, 47, 63, 57, 78, 50, 29, 35, 60, 74, 31, 41, 6, 22, 67, 40, 45, 76, 37, 55, 18, 52, 39, 66, 33, 11, 27, 7, 43, 24, 14, 28, 53, 9, 12, 75, 15, 10, 8, 5, 30, 16, 3, 38, 2, 4, 68, 49, 59, 62, 23, 13, 72, 61, 73, 19, 69, 54, 17, 58, 25]
init_cost = cost_function(init_sol, problem) + rejected_calls_cost(init_sol, problem)
sols_iter = []
costs_iter = []
for i in range(1):
    sols, costs = run_problem(problem, init_sol)
    sols_iter.append(sols)
    costs_iter.append(costs)

best_local_1opt = min([op[0] for op in costs_iter])
print("Avg Local 1-opt", np.mean([op[0] for op in costs_iter]))
print("Best Local 1-opt", best_local_1opt)
print("Improvement%", (init_cost - best_local_1opt) / 100)

best_local_2opt = min([op[1] for op in costs_iter])
print("Avg Local 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Local 2-opt", best_local_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_local_3opt = min([op[2] for op in costs_iter])
print("Avg Local 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Local 3-opt", best_local_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 


best_anneal_1opt = min([op[0] for op in costs_iter])
print("Avg Anneal 1-opt", np.mean([op[0]for op in costs_iter]))
print("Best Anneal 1-opt", best_anneal_1opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_2opt = min([op[1] for op in costs_iter])
print("Avg Anneal 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Anneal 2-opt", best_anneal_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_3opt = min([op[2] for op in costs_iter])
print("Avg Anneal 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Anneal 3-opt", best_anneal_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 

## C130V40

In [ ]:
problem = load_problem('./Call_130_Vehicle_40')
init_sol = [0, 0, 0, 30, 30, 0, 22, 22, 0, 25, 25, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 21, 21, 0, 76, 76, 0, 0, 0, 0, 0, 0, 78, 78, 0, 0, 0, 35, 35, 0, 0, 0, 40, 40, 0, 0, 0, 37, 44, 102, 118, 94, 126, 4, 14, 82, 88, 71, 70, 63, 17, 77, 47, 68, 111, 51, 67, 3, 125, 75, 86, 10, 108, 87, 15, 104, 80, 42, 29, 105, 27, 97, 103, 23, 130, 95, 46, 11, 64, 59, 43, 32, 73, 45, 116, 69, 122, 127, 65, 121, 53, 109, 93, 49, 62, 54, 83, 48, 66, 119, 113, 19, 107, 7, 92, 85, 115, 57, 55, 36, 79, 12, 128, 2, 123, 84, 6, 24, 60, 101, 9, 120, 33, 20, 99, 13, 117, 74, 110, 39, 56, 106, 52, 26, 90, 8, 112, 96, 89, 114, 41, 81, 61, 98, 38, 31, 124, 100, 91, 16, 34, 50, 58, 18, 5, 28, 72, 129]
init_cost = cost_function(init_sol, problem) + rejected_calls_cost(init_sol, problem)
sols_iter = []
costs_iter = []
for i in range(1):
    sols, costs = run_problem(problem, init_sol)
    sols_iter.append(sols)
    costs_iter.append(costs)

best_local_1opt = min([op[0] for op in costs_iter])
print("Avg Local 1-opt", np.mean([op[0] for op in costs_iter]))
print("Best Local 1-opt", best_local_1opt)
print("Improvement%", (init_cost - best_local_1opt) / 100)

best_local_2opt = min([op[1] for op in costs_iter])
print("Avg Local 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Local 2-opt", best_local_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_local_3opt = min([op[2] for op in costs_iter])
print("Avg Local 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Local 3-opt", best_local_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 


best_anneal_1opt = min([op[0] for op in costs_iter])
print("Avg Anneal 1-opt", np.mean([op[0]for op in costs_iter]))
print("Best Anneal 1-opt", best_anneal_1opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_2opt = min([op[1] for op in costs_iter])
print("Avg Anneal 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Anneal 2-opt", best_anneal_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_3opt = min([op[2] for op in costs_iter])
print("Avg Anneal 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Anneal 3-opt", best_anneal_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 

## C300V90

In [ ]:
problem = load_problem('./Call_300_Vehicle_90')
init_sol = [0, 226, 226, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 290, 290, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 73, 73, 0, 0, 0, 0, 287, 287, 0, 0, 0, 0, 0, 0, 0, 187, 187, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 34, 34, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 86, 86, 0, 0, 0, 160, 160, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 271, 41, 255, 36, 40, 204, 150, 84, 153, 218, 244, 260, 44, 79, 219, 169, 178, 240, 227, 165, 212, 39, 12, 55, 29, 264, 298, 59, 62, 13, 141, 230, 225, 234, 51, 202, 75, 286, 272, 116, 119, 54, 26, 174, 64, 300, 49, 192, 65, 97, 156, 60, 23, 48, 203, 101, 47, 189, 195, 106, 46, 145, 233, 22, 56, 183, 130, 280, 179, 184, 159, 282, 89, 25, 182, 9, 128, 162, 68, 108, 143, 293, 50, 214, 254, 103, 63, 28, 105, 294, 74, 37, 263, 92, 205, 18, 284, 87, 148, 191, 239, 216, 121, 8, 185, 133, 289, 283, 16, 278, 53, 258, 99, 249, 242, 175, 215, 4, 299, 3, 69, 118, 209, 7, 223, 224, 200, 109, 193, 201, 57, 102, 42, 220, 180, 140, 124, 19, 104, 199, 117, 114, 167, 88, 221, 129, 2, 291, 265, 149, 275, 247, 173, 292, 208, 82, 197, 120, 252, 177, 125, 257, 168, 158, 142, 32, 80, 137, 67, 152, 181, 161, 296, 77, 279, 35, 31, 112, 15, 256, 71, 126, 288, 172, 98, 6, 70, 94, 139, 245, 171, 155, 78, 115, 30, 134, 232, 273, 157, 20, 268, 243, 90, 144, 250, 229, 248, 132, 5, 198, 38, 154, 196, 297, 269, 91, 237, 17, 45, 110, 276, 261, 259, 190, 211, 194, 11, 52, 111, 213, 206, 253, 295, 274, 76, 138, 186, 207, 33, 131, 222, 236, 251, 24, 72, 1, 151, 113, 231, 127, 188, 210, 163, 81, 66, 14, 135, 83, 21, 27, 217, 147, 262, 96, 93, 238, 170, 270, 241, 281, 164, 122, 10, 235, 61, 95, 123, 266, 246, 85, 43, 176, 277, 100, 107, 166, 285, 136, 58, 228, 267, 146]
init_cost = cost_function(init_sol, problem) + rejected_calls_cost(init_sol, problem)
sols_iter = []
costs_iter = []
for i in range(1):
    sols, costs = run_problem(problem, init_sol)
    sols_iter.append(sols)
    costs_iter.append(costs)

best_local_1opt = min([op[0] for op in costs_iter])
print("Avg Local 1-opt", np.mean([op[0] for op in costs_iter]))
print("Best Local 1-opt", best_local_1opt)
print("Improvement%", (init_cost - best_local_1opt) / 100)

best_local_2opt = min([op[1] for op in costs_iter])
print("Avg Local 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Local 2-opt", best_local_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_local_3opt = min([op[2] for op in costs_iter])
print("Avg Local 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Local 3-opt", best_local_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 


best_anneal_1opt = min([op[0] for op in costs_iter])
print("Avg Anneal 1-opt", np.mean([op[0]for op in costs_iter]))
print("Best Anneal 1-opt", best_anneal_1opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_2opt = min([op[1] for op in costs_iter])
print("Avg Anneal 2-opt", np.mean([op[1] for op in costs_iter]))
print("Best Anneal 2-opt", best_anneal_2opt)
print("Improvement%", (init_cost - best_local_2opt) / 100) 

best_anneal_3opt = min([op[2] for op in costs_iter])
print("Avg Anneal 3-opt", np.mean([op[2] for op in costs_iter]))
print("Best Anneal 3-opt", best_anneal_3opt)
print("Improvement%", (init_cost - best_local_3opt) / 100) 